In [1]:
import os
import sys
import sqlite3

# 실습 0-6 해답이후 수정

In [12]:
try:
    conn = sqlite3.connect('Data/sqlite2_db.db')
    curs = conn.cursor()
    
    # table 생성
    sql = """create table if not exists item(
    code integer primary key,
    name text(30) unique not null,
    qty integer default 0,
    unit_price real default 0.0
    )"""
    curs.execute(sql)
    conn.commit()
    
    # 상품등록 (중복처리)
    while True:
        in_code = int(input('상품코드 입력: '))
        sql = f'select * from item where code = {in_code}'
        curs.execute(sql)
        rows = curs.fetchall()
        
        if len(rows) > 0:    # len(rows) > 0 초과로 있나? # if rows: 또는 if True: 또는 if not rows:
            print("존재하는 코드입니다")
            os.system('pause')  # 터미널에 직접 일시정지 명령, 아무렇게나 눌렀을 때 다음 칸으로 출력이 됨
            # 계속하려면 아무 키나 누르십시오. 혹은 계속하려면 <Enter> 키를 누르십시오.:와 같은 메시지 출력됨
            os.system('cls')    # python 화면 지우기: 터미널 창에서 지움. jupyter에서는 실행 안 됨
            continue
        else:
            in_name = input('상품명 입력: ')
            in_qty = int(input('수량 입력: '))
            in_unit_price = int(input('단가 입력: '))
            sql = f'insert into item(code, name, qty, unit_price) values({in_code}, "{in_name}", {in_qty}, {in_unit_price})'
            curs.execute(sql)
            conn.commit()
            print("상품 등록을 성공했습니다")
            break
    
    # 모든 레코드 조회
    curs.execute('select * from item')
    rows = curs.fetchall()
    print('======= 상품조회1 ======')
    print("(code | name | qty | price)")
    for row in rows:
        print(row)
        
    # 단일 레코드 조회
    in_code = int(input('조회할 상품의 코드를 입력하세요: '))
    sql = f'select * from item where code = {in_code}'
    curs.execute(sql)
    rows = curs.fetchall()
    if rows:
        for row in rows:
            print("[조회 결과] 코드: {}, 제품명: {}, 수량: {}, 단가: {} 입니다.".format(row[0], row[1], row[2], row[3]))
    else:
        print("조회 결과, 코드 {}에 해당하는 상품은 없습니다".format(in_code))

except Exception as e:
    print("DB연동 실패: ", e)
    conn.rollback()

finally:
    curs.close()
    conn.close()           
            

상품코드 입력: 8
존재하는 코드입니다
상품코드 입력: 9
상품명 입력: 노트북
수량 입력: 1
단가 입력: 1300
상품 등록을 성공했습니다
======= 상품조회1 ======
(code | name | qty | price)
(1, '선풍기', 1, 150.0)
(2, '에어컨', 1, 200.0)
(3, '충전기', 1, 100.0)
(4, '키보드', 1, 70.0)
(5, '마우스', 1, 60.0)
(6, '모니터', 1, 120.0)
(7, '휴대폰', 1, 300.0)
(8, '프린터', 1, 170.0)
(9, '노트북', 1, 1300.0)
조회할 상품의 코드를 입력하세요: 9
[조회 결과] 코드: 9, 제품명: 노트북, 수량: 1, 단가: 1300.0 입니다.


# [실습2] MySQL p295-304

- IP 이해 필요
    - TCP/IP. layer의 부속품. 계층에 관한 이해가 필요
    - 네트워크 기초. OS 7 layer. Internet 5 layer 호환되는 기능 
    - 포트: 자료를 주고받을 때 고속도로 버스 ?
- 동영상에서 user를 root1234로 할 필요 없음
- portNumber: 3306 (MySQL에서는 주로 3306 사용)    

### SQL을, CLI, GUI 방식 둘 다 사용할 줄 알아야 함
- SQL Stored Query Language. 

### (CLI) 'mySQL command line client' 실행

    (비밀번호 입력)

    show databases;

    ##### p302 데이터베이스 생성
    #'work'라는 database만들어라
    create database work;   

    # 데이터베이스 보기
    show databases;         

    # database 선택
    use work;               
    
    # table 'goods' 만들기
    create table goods(    
    code int primary key,
    name varchar(20) not null,
    su int,
    dan int);

    # table 보기
    show tables;
    
    # 레코드 추가하기
    insert into goods values(1, '냉장고', 2, 850000);
    insert into goods values(2, '세탁기', 3, 550000);
    insert into goods values(3, '전자레인지', 2, 350000);
    
    # 레코드 조회하기
    select * from goods;
    
    # 종료
    quit
    
### (GUI) MySQL workbench 실행
    
    테이블 확인

# [실습3] p303~305

### (cmd창)
    pip install pymysql
    
### (VS코드)
    import pymysql
    print(pymysql.version_info)
    
    ##### DB환경변수 선언
    config = {                  # MySQL데이터베이스를 사용하기 위해 환경변수를 딕셔너리로 생성
        'host' : '127.0.0.1',   # MySql 이 동작하는 ipv4주소: DB가 동작중인 컴퓨터의 IP주소 
        'user' : 'root',        # MySql 설치할 때 정한 계정 
        'passwd' : 'root1234',  # MySql 설치할 때 정한 비밀번호
        'database' : 'work',    # 데이터베이스 (not table)
        'port' : 3306,          # MySql이 응용프로그램과 통신할 때 사용하는 포트번호. 정수값
        'charset' : 'utf8',     # 한글을 사용하겠다는 설정 
        'use_unicode' : True    # 한글을 사용하겠다는 설정 
        }
        
    ##### DB와 연결 및 cursor 객체를 설정
    # (1) db 연동 객체 
    conn = pymysql.connect(**config)    # 딕셔너리 config를 인수로 사용하여 conn 객체를 만듦.

    # sql 실행 객체 
    cursor = conn.cursor()              # conn 객체로부터 cursor() 메소드를 호출하여 cursor 참조변수를 만듦

# [실습4] p305~306

### (VS코드)에서 실행

# [실습5] 

### (VS코드)에서 실행

In [ ]:
### (VS코드) setting.json 설정 변경

{
    "editor.fontSize": 15,
    "editor.fontWeight": 1000,
    "code-runner.saveFileBeforeRun": true,
    "code-runner.runInTerminal": true,
    "python.linting.enabled": false,
    "python.pythonPath": "C:\\Program Files\\python\\Python311",  ## python.exe 설치된 위치로 바꿀 것
    "window.zoomLevel": 0,
    "terminal.integrated.fontSize": 15,
    "code-runner.clearPreviousOutput": true,
    "explorer.confirmDelete": false,
    "explorer.incrementalNaming": "smart",
    "files.autoSave": "afterDelay",
    "command": "workbench.action.terminal.clear",
    "workbench.startupEditor": "newUntitledFile",
    "workbench.colorTheme": "Visual Studio Light",
    "jupyter.alwaysTrustNotebooks": true,
    "jupyter.askForKernelRestart": false,
    "liveServer.settings.donotShowInfoMsg": true,
    "workbench.editorAssociations": {
        "*.ipynb": "jupyter-notebook"
    },
    "python.showStartPage": false,
    "python.languageServer": "Pylance",
    "notebook.cellToolbarLocation": {
        "default": "right",
        "jupyter-notebook": "left"
    },
    "python.defaultInterpreterPath": "C:\\Program Files\\python\\Python311",  ## python.exe 설치된 위치로 바꿀 것
    "editor.minimap.enabled": false,
    "terminal.integrated.scrollback": 1000,
    "editor.fontFamily": "D2 Coding ligature, D2Coding, Consolas, 'Courier New', monospace",
    "editor.fontLigatures": true,
    "editor.mouseWheelZoom": true,
    "notebook.lineNumbers": "on",
    "explorer.confirmDelete": false,
}

## 실습5 분석하기

### def로 함수를 정의한 후 정의한 함수 활용한 코드 실행

###  if __name__ == "__main__" : 시작점 지정하는 문법

    - '__name__'이라는 변수의 값이 '__main__'이면 하단의 코드를 실행하라는 뜻
    - C언어나 자바와 같은 프로그래밍 언어에서는 main()이라는 함수를 시작으로 프로그램을 실행시킴
    - 파이썬은 main문이 없는 대신에, 들여쓰기가 되지 않은 Level0의 코드를 가장 먼저 실행시킴
    
https://hyoje420.tistory.com/45

    - 'module.py'
        def hello():
            print("Hello!")
        print(__name__)
    - 'main.py'
        import module
        print(__name__)
        module.hello()

    - 직접 실행한 파일(module.py)은, '__name__' 출력 시, '__main__'이라는 값을 출력하고
    - 그 모듈을 import하면(main.py) '__name__' 출력 시, 모듈 이름(module)과 '__main__'이 출력됨

    - if __name__=="__main__"이라는 조건문을 넣어주면, 모듈이름(module)은 출력되지 않음
    - 'module.py' 
        def hello():
            print("Hello!")
        if __name__=="__main__":
            print(__name__)

    - if __name__=="__main__"이라는 조건문 아래에는 직접 실행시켰을 때만 실행되길 원하는 코드를 넣어 실행